## Workshop TP3

## 1) EDA

## 2) Defino X, Y 

## 3) Split Train, Test (Utilizar shuffle = True)

## 4) Derivo las variables (dummies/one hot encoder, scaling)

## 5) Instancio el modelo (con train)

## 6) Entrenar el modelo

## 7) Performance

**Test**

Luego tenés que escalar y transformar el test

In [1]:
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn import metrics
import statsmodels.api as sm
from statsmodels.tools import eval_measures

In [2]:
data_CABA = pd.read_csv('properati_CABA.csv', sep=",")

In [3]:
data_CABA.rename(columns = {'price_m2_calc_supTotal': 'price_m2', 'surface_total_in_m2_CLEAN': 'surface'}, inplace = True)

In [4]:
data_CABA.head()

,Unnamed: 0,property_type,place_name,state_name,price_aprox_usd,rooms,contrafrente,cochera,amenities,esp_exterior,mas_1banio,estrenar,surface,price_m2
0,0,PH,Mataderos,Capital Federal,62000.0,2.0,0,0,0,1,1,0,55.0,1127.272727
1,2,apartment,Mataderos,Capital Federal,72000.0,2.0,0,0,0,0,1,0,55.0,1309.090909
2,3,PH,Liniers,Capital Federal,95000.0,3.0,0,0,0,1,0,0,57.5,1652.173913
3,6,apartment,Belgrano,Capital Federal,138000.0,1.0,0,0,1,0,1,1,45.0,3066.666667
4,7,apartment,Belgrano,Capital Federal,195000.0,2.0,0,0,1,0,1,1,65.0,3000.000000


#### Definimos X e Y

In [5]:
X = data_CABA.drop(['price_aprox_usd', 'price_m2', 'state_name'], axis = 1)
y = data_CABA.price_m2

In [6]:
data_CABA.shape

(29260, 14)

In [7]:
y = data_CABA['price_m2']
X = data_CABA[['surface', 'place_name', 'property_type']]

#### Dividimos en Train y Test

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 13)

In [9]:
data_CABA[['price_m2','surface', 'place_name', 'property_type']].corr()

,price_m2,surface
price_m2,1.000000,-0.135758
surface,-0.135758,1.000000


#### Obtenemos Dummies

In [10]:
X_train_dummies = pd.get_dummies(X_train, drop_first = True).reset_index(drop = True)

In [11]:
y_train = y_train.reset_index(drop = True)

#### Version alternativa OneHot

In [12]:
from sklearn.preprocessing import OneHotEncoder 
onehot_encoder2 = OneHotEncoder(drop='first')
dummy_oneHot_correct = onehot_encoder2.fit_transform(X_train[['place_name', 'property_type']])
dummy_oneHot_correct = pd.DataFrame(dummy_oneHot_correct.toarray())
dummy_oneHot_correct.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


#### Stats Model

In [13]:
X_stats = sm.add_constant(X_train_dummies)

model = sm.OLS(y_train, X_stats).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               price_m2   R-squared:                       0.609
Model:                            OLS   Adj. R-squared:                  0.608
Method:                 Least Squares   F-statistic:                     615.2
Date:                Tue, 24 Aug 2021   Prob (F-statistic):               0.00
Time:                        20:24:25   Log-Likelihood:            -1.8233e+05
No. Observations:               23408   AIC:                         3.648e+05
Df Residuals:                   23348   BIC:                         3.653e+05
Df Model:                          59                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                            1792.1287     66.834     26.815      0.000    1661.129    1923.128
surface                            -1.0013      0.035    -28.520      0.000      -1.070      -0.932
place_name_Agronomía              -51.1874    119.552     -0.428      0.669    -285.516     183.141
place_name_Almagro                -57.3338     67.989     -0.843      0.399    -190.596      75.928
place_name_Balvanera             -322.0802     69.929     -4.606      0.000    -459.145    -185.016
place_name_Barracas              -292.4122     72.550     -4.030      0.000    -434.616    -150.209
place_name_Barrio Norte           598.0880     68.089      8.784      0.000     464.629     731.547
place_name_Belgrano               727.8311     66.207     10.993      0.000     598.062     857.601
place_name_Boca                  -556.3177     78.923     -7.049      0.000    -711.013    -401.623
place_name_Boedo                 -168.0854     69.389     -2.422      0.015    -304.093     -32.078
place_name_Caballito              193.8614     66.497      2.915      0.004      63.523     324.200
place_name_Centro / Microcentro  -151.9842     79.439     -1.913      0.056    -307.690       3.721
place_name_Chacarita                4.6556     79.213      0.059      0.953    -150.608     159.919
place_name_Coghlan                327.0421     76.519      4.274      0.000     177.059     477.025
place_name_Colegiales             302.9209     74.053      4.091      0.000     157.773     448.069
place_name_Congreso              -436.7298     75.609     -5.776      0.000    -584.928    -288.532
place_name_Constitución          -459.5833     79.945     -5.749      0.000    -616.281    -302.885
place_name_Flores                -139.4524     67.427     -2.068      0.039    -271.613      -7.291
place_name_Floresta              -404.1837     71.325     -5.667      0.000    -543.986    -264.382
place_name_Las Cañitas           1120.8492     87.100     12.869      0.000     950.127    1291.571
place_name_Liniers               -284.0824     72.666     -3.909      0.000    -426.513    -141.652
place_name_Mataderos             -430.8588     72.559     -5.938      0.000    -573.080    -288.637
place_name_Monserrat             -137.6890     71.186     -1.934      0.053    -277.217       1.839
place_name_Monte Castro          -129.8661    101.186     -1.283      0.199    -328.197      68.464
place_name_Nuñez                  635.3484     69.501      9.142      0.000     499.122     771.575
place_name_Once                  -486.5543     81.600     -5.963      0.000    -646.495    -326.613
place_name_Palermo                909.0624     65.886     13.798      0.000     779.922    1038.202
place_name_Parque Avellaneda     -475.1502    111.360     -4.267      0.000    -693

### Tiro los que no pasan el test "p.value"

In [14]:
X_train_dummies_pvalue = X_train_dummies.drop(["place_name_Villa General Mitre","place_name_Agronomía","place_name_Almagro","place_name_Chacarita","place_name_Centro / Microcentro","place_name_Monte Castro","place_name_Monserrat","place_name_Parque Centenario","place_name_Parque Chas","place_name_San Nicolás","place_name_San Telmo","place_name_Versalles","place_name_Villa Devoto","place_name_Villa Ortuzar","place_name_Villa Pueyrredón","place_name_Villa Santa Rita","place_name_Villa del Parque","property_type_house"], axis=1)

In [15]:
X_stats = sm.add_constant(X_train_dummies_pvalue)

model = sm.OLS(y_train, X_stats).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               price_m2   R-squared:                       0.607
Model:                            OLS   Adj. R-squared:                  0.607
Method:                 Least Squares   F-statistic:                     881.4
Date:                Tue, 24 Aug 2021   Prob (F-statistic):               0.00
Time:                        20:24:25   Log-Likelihood:            -1.8237e+05
No. Observations:               23408   AIC:                         3.648e+05
Df Residuals:                   23366   BIC:                         3.652e+05
Df Model:                          41                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                         1766.4355     16.158    109.323      0.000    1734.765    1798.106
surface                         -0.9990      0.034    -29.459      0.000      -1.065      -0.933
place_name_Balvanera          -298.9304     27.438    -10.895      0.000    -352.711    -245.150
place_name_Barracas           -269.8634     33.584     -8.035      0.000    -335.691    -204.036
place_name_Barrio Norte        621.0463     22.300     27.849      0.000     577.336     664.757
place_name_Belgrano            750.3903     15.602     48.097      0.000     719.810     780.970
place_name_Boca               -533.9783     45.764    -11.668      0.000    -623.678    -444.279
place_name_Boedo              -145.3724     26.038     -5.583      0.000    -196.408     -94.336
place_name_Caballito           216.4996     16.823     12.869      0.000     183.525     249.474
place_name_Coghlan             349.6787     41.486      8.429      0.000     268.364     430.994
place_name_Colegiales          325.5555     36.727      8.864      0.000     253.568     397.543
place_name_Congreso           -413.4595     39.764    -10.398      0.000    -491.400    -335.519
place_name_Constitución       -436.4961     47.524     -9.185      0.000    -529.646    -343.346
place_name_Flores             -116.6695     20.226     -5.768      0.000    -156.314     -77.025
place_name_Floresta           -381.6439     30.858    -12.368      0.000    -442.128    -321.160
place_name_Las Cañitas        1143.5409     58.769     19.458      0.000    1028.349    1258.733
place_name_Liniers            -261.2605     33.873     -7.713      0.000    -327.653    -194.868
place_name_Mataderos          -408.3153     33.663    -12.130      0.000    -474.296    -342.334
place_name_Nuñez               657.8494     26.307     25.007      0.000     606.286     709.413
place_name_Once               -462.8864     50.251     -9.211      0.000    -561.382    -364.391
place_name_Palermo             931.8946     14.208     65.591      0.000     904.046     959.743
place_name_Parque Avellaneda  -452.8694     91.025     -4.975      0.000    -631.285    -274.454
place_name_Parque Chacabuco   -280.4576     59.125     -4.743      0.000    -396.347    -164.569
place_name_Parque Patricios   -290.6253     37.244     -7.803      0.000    -363.627    -217.624
place_name_Paternal           -224.3708     51.508     -4.356      0.000    -325.329    -123.413
place_name_Pompeya            -814.7752     82.843     -9.835      0.000    -977.153    -652.397
place_name_Puerto Madero      3549.2761     27.830    127.534      0.000    3494.727    3603.825
place_name_Recoleta            998.0590     19.556     51.035      0.000     959.727    1036.391
place_name_Retiro              412.1312     45.549      9.048      0.000    

#### Entreno Lasso

In [16]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

In [17]:
x_train_sup_sc = sc.fit_transform(X_train_dummies_pvalue['surface'].values.reshape(-1,1))

In [18]:
x_train_sup_sc = pd.DataFrame(x_train_sup_sc, columns = ['surface'])

In [19]:
X_train_dummies['surface'] = x_train_sup_sc

In [20]:
# Las dummies no hace falta que se reescalen

In [21]:
import numpy as np

alphas = np.linspace(0.01, 100, 50) #Aquí busco el hiperparámetro


In [22]:
model = linear_model.LassoCV(alphas = alphas, cv = 5)

In [23]:
cv = KFold(5, shuffle=True)
lassocv = linear_model.LassoCV(alphas=np.linspace(0.001,1, 100), cv=cv, normalize=True)
lassocv.fit(X_train_dummies, y_train)
predictions = lassocv.predict(X_train_dummies)

print ("\nr^2:", lassocv.score(X_train_dummies, y_train))
print ("\nalpha:", lassocv.alpha_)
print ("\ncoeficientes:", lassocv.coef_)
print ("\nintercepto:", lassocv.intercept_)
print ("\nzero_coefs:", (lassocv.coef_ == 0).sum())


r^2: 0.6083538932790202

alpha: 0.011090909090909092

coeficientes: [ -140.39134456   -16.5760511    -57.22838809  -319.52140029
  -287.59810703   580.31447338   713.07333163  -546.07555799
  -165.31114105   179.10972859  -137.02855564    -0.
   300.50258208   278.4669786   -427.60354391  -447.36343825
  -140.21382093  -400.38998705  1085.38032132  -278.71060014
  -426.20786951  -132.62392458  -104.2804191    615.27680222
  -472.96189422   895.69181895  -445.22312381     0.
  -286.45857154  -179.60288034  -306.68385925  -233.50664541
  -812.59810573  3504.26222121   958.31275356   360.15662633
   145.45099078  -251.66877808   -76.4635227     43.63183715
  -315.22142669  -211.50701921   -83.50605611   225.96572493
    30.97074812  -147.75369598  -769.79115248  -173.83971532
    58.3969228     30.8645067   -235.28953279  -171.16468498
 -1170.83125163   212.59609267    -0.           539.30913947
   -14.55796164  9741.8326432    729.99344751]

intercepto: 1705.4044719017543

zero_coefs: 3

In [24]:
pd.DataFrame([[v, c] for v,c in zip(X_train_dummies.columns, lassocv.coef_)]).head()

,0,1
0,surface,-140.391345
1,place_name_Agronomía,-16.576051
2,place_name_Almagro,-57.228388
3,place_name_Balvanera,-319.521400
4,place_name_Barracas,-287.598107


#### Regularizo a Stats Models para ver quecomo queda

In [25]:
best_alpha = lassocv.alpha_

no_reg_model = sm.OLS(y_train, X_stats)

reg_model = no_reg_model.fit_regularized(alpha = best_alpha, L1_wt = 1)

In [26]:
predictions = reg_model.predict(X_stats)

In [27]:
#metrics.r2_score(X_stats, predictions)

#### Entreno RidgeCV

In [28]:
cv = KFold(5, shuffle=True)
RidgeCV = linear_model.RidgeCV(alphas=np.linspace(0.001,1, 100), cv=cv, normalize=True)
RidgeCV.fit(X_train_dummies, y_train)
predictions = RidgeCV.predict(X_train_dummies)

print ("\n r^2:", RidgeCV.score(X_train_dummies, y_train))
print ("\n alpha:", RidgeCV.alpha_)
print ("\n coeficientes:", RidgeCV.coef_)
print ("\n intercepto:", RidgeCV.intercept_)


 r^2: 0.6082409650613709

 alpha: 0.011090909090909092

 coeficientes: [-1.40441567e+02 -2.83734508e+02 -2.87692111e+02 -5.49669076e+02
 -5.21062456e+02  3.60866068e+02  4.89156315e+02 -7.81817713e+02
 -3.96258150e+02 -3.90202749e+01 -3.77762994e+02 -2.26307475e+02
  9.24195332e+01  6.83984758e+01 -6.61771488e+02 -6.85529236e+02
 -3.68899206e+02 -6.31749997e+02  8.76927168e+02 -5.13601889e+02
 -6.59701186e+02 -3.66038011e+02 -3.60808770e+02  3.97344110e+02
 -7.11379689e+02  6.69085743e+02 -7.04498111e+02 -2.20759856e+02
 -5.31582513e+02 -4.46100625e+02 -5.42222602e+02 -4.76810347e+02
 -1.06271639e+03  3.25611718e+03  7.33556002e+02  1.53765979e+02
 -6.63813222e+01 -4.84094926e+02 -3.19287739e+02 -1.70599608e+02
 -5.64280708e+02 -4.81778735e+02 -3.44897586e+02  8.62873471e+00
 -1.78014005e+02 -4.09653601e+02 -1.00415451e+03 -4.08308661e+02
 -1.31206595e+02 -1.70945272e+02 -5.08388073e+02 -4.31118604e+02
 -1.48614201e+03 -4.74191677e+00 -2.28381508e+02  5.28079678e+02
 -2.95693999e+01  

In [29]:
#print ("\nr^2 test:", RidgeCV.score(X_test, y_test))

# me arroja "could not convert string to float: 'Recoleta'"  ??????????????????????????????

#### Elastic Net

In [30]:
elastic_net = linear_model.ElasticNet(alpha=0.011, normalize=True)
elastic_net.fit(X_train_dummies, y_train)
predictions = elastic_net.predict(X_train_dummies)

print ("\n r^2:", elastic_net.score(X_train_dummies, y_train))
print ("\n coeficientes:", elastic_net.coef_)
print ("\n intercepto:", elastic_net.intercept_)


 r^2: 0.011329673943634821

 coeficientes: [ -0.97103559  -3.47258922  -2.78791013  -4.76639719  -5.30625713
   2.57678893   3.55483856  -7.24202355  -3.53526056  -0.74492525
  -2.81592558  -2.22881783  -0.05831114  -0.2981479   -5.18798512
  -5.7563886   -3.82187602  -6.37998386   6.29120986  -5.65288138
  -8.06693842  -3.21107451  -4.43137273   2.33051785  -5.72152462
   5.40279541  -8.22091561  -2.15647104  -6.31392556  -3.81620508
  -5.23321596  -5.10733507 -11.95554038  25.19934346   5.58192673
   0.7215493   -1.40827997  -4.35477166  -2.80652276  -1.58666912
  -4.13530727  -7.03973895  -5.78208379  -0.29112736  -3.33571832
  -4.82161447 -10.70911787  -4.1795966   -2.53672757  -3.17563041
  -7.53808913  -5.23441089 -14.72157242  -0.52536505  -2.79676662
   6.09027675  -7.97298756 -17.52210512   0.20756905]

 intercepto: 2499.697810856898


In [31]:
from sklearn.metrics import r2_score

In [32]:
# Hacer funciones que reescalen

In [33]:
# Hacer un DataFrame que compare los distintos modelos 

#### Probando con Surface de distintos grados  (da mejroes resultados)

In [34]:
X_train_dummies["surface2"] = X_train_dummies.surface * X_train_dummies.surface

cv = KFold(5, shuffle=True)
lassocv = linear_model.LassoCV(alphas=np.linspace(0.001,1, 1000), cv=cv, normalize=True)
lassocv.fit(X_train_dummies, y_train)
predictions = lassocv.predict(X_train_dummies)

print ("\nr^2:", lassocv.score(X_train_dummies, y_train))
print ("\nalpha:", lassocv.alpha_)
print ("\nzero_coefs:", (lassocv.coef_ == 0).sum())


r^2: 0.6082865329846844

alpha: 0.031

zero_coefs: 7


In [35]:
X_train_dummies["surface2"] = X_train_dummies.surface * X_train_dummies.surface
X_train_dummies["surface3"] = X_train_dummies.surface2 * X_train_dummies.surface 

cv = KFold(5, shuffle=True)
lassocv = linear_model.LassoCV(alphas=np.linspace(0.001,1, 1000), cv=cv, normalize=True)
lassocv.fit(X_train_dummies, y_train)
predictions = lassocv.predict(X_train_dummies)

print ("\nr^2:", lassocv.score(X_train_dummies, y_train))
print ("\nalpha:", lassocv.alpha_)
print ("\nzero_coefs:", (lassocv.coef_ == 0).sum())


r^2: 0.6101961570771363

alpha: 0.001

zero_coefs: 2


In [36]:
X_train_dummies["surface2"] = X_train_dummies.surface * X_train_dummies.surface
X_train_dummies["surface3"] = X_train_dummies.surface2 * X_train_dummies.surface 
X_train_dummies["surface4"] = X_train_dummies.surface3 * X_train_dummies.surface 

cv = KFold(5, shuffle=True)
lassocv = linear_model.LassoCV(alphas=np.linspace(0.001,1, 1000), cv=cv, normalize=True)
lassocv.fit(X_train_dummies, y_train)
predictions = lassocv.predict(X_train_dummies)

print ("\nr^2:", lassocv.score(X_train_dummies, y_train))
print ("\nalpha:", lassocv.alpha_)
print ("\nzero_coefs:", (lassocv.coef_ == 0).sum())

C:\Users\Dani\anaconda3\envs\dhdsblend2021\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 74996549.4697771, tolerance: 1630142.9988088566
  model = cd_fast.enet_coordinate_descent_gram(



r^2: 0.6101305015327514

alpha: 0.004

zero_coefs: 5
